In [1]:
%matplotlib inline

In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import zscore
import statistics as stats
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression


In [3]:
# read sales data as 'df'
df = pd.read_csv('wallmart.csv')

In [4]:
df.head()

Year  Month ProductCategory  Sales(In ThousandDollars)
0  2009      1   WomenClothing                     1755.0
1  2009      1     MenClothing                      524.0
2  2009      1   OtherClothing                      936.0
3  2009      2   WomenClothing                     1729.0
4  2009      2     MenClothing                      496.0

In [5]:
df.shape

(216, 4)

In [6]:
#read excel file with multiple sheets
file = pd.ExcelFile('weather.xlsx')
file.sheet_names

['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [7]:
# checking shape of each sheet
for i in file.sheet_names:
    data = pd.read_excel(file, sheet_name= i )
    print(data.shape)


(365, 23)
(365, 23)
(365, 23)
(366, 23)
(365, 23)
(365, 23)
(365, 23)
(366, 23)


In [8]:
# read weather data excel sheet into df_weather
df_weather = pd.read_excel('weather.xlsx', sheetname=None, ignore_index=True)

C:\Users\noah\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [9]:
# concat all sheets into one
cdf_weather = pd.concat(df_weather.values())

cdf_weather.head()

Year Month  Day Temp high (°C) Temp avg (°C) Temp low (°C)  \
0  2009   Jan    1             -3            -6            -9   
1  2009   Jan    2              1            -2            -5   
2  2009   Jan    3              3             1            -2   
3  2009   Jan    4              6             1            -4   
4  2009   Jan    5              6             5             3   

  Dew Point high (°C) Dew Point avg (°C) Dew Point low (°C) Humidity (%) high  \
0                 -16                -17                -19                54   
1                  -3                 -7                -17                78   
2                  -5                 -9                -13                72   
3                 -10                -12                -13                55   
4                  -1                 -5                -16                62   

      ...      Sea Level Press. (hPa) avg Sea Level Press. (hPa) low  \
0     ...                            1023                       1015   
1     ...                            1012                       1007   
2     ...                            1015                       1008   
3     ...                            1017                       1015   
4     ...                            1014                       1013   

  Visibility (km) high Visibility (km) avg Visibility (km) low  \
0                   16                  16                  16   
1                   16                  13                   2   
2                   16                  16                  16   
3                   16                  16                  16   
4                   16                  16                  16   

  Wind (km/h) low Wind (km/h) avg Wind (km/h) high Precip. (mm) sum  \
0              37              18               60                0   
1              27              10               48                T   
2              27              16               42                T   
3              32              12               40                0   
4              23              11               34                T   

  WeatherEvent  
0          NaN  
1         Snow  
2          NaN  
3          NaN  
4          NaN  

[5 rows x 23 columns]

# Preprocessing sales data

In [10]:
df.rename(columns = {'Sales(In ThousandDollars)' : 'Sales'}, inplace= True)

In [11]:
df['ProductCategory'] = df['ProductCategory'].astype('category', categories = df['ProductCategory'].unique( ))

C:\Users\noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [12]:
df['ProductCategory'].replace(['OtherClothing','MenClothing','WomenClothing'] , [3,2,1], inplace = True)

In [13]:
df.isna().apply(pd.value_counts)

Year  Month  ProductCategory  Sales
False  216.0  216.0            216.0    204
True     NaN    NaN              NaN     12

In [14]:
# simple loop to replace NaN values with mean of adjacent sales of same category
x = []
for i in range(0,len(df.Sales)):
    if df.isna().Sales[i] == True :
        x.append(stats.mean([df.Sales[i+3],df.Sales[i-3]]))
        
    else:
        x.append(df.Sales[i])
Sales = pd.Series(x)


In [15]:
df['Sales'] = Sales

In [16]:
df.isna().apply(pd.value_counts)

Year  Month  ProductCategory  Sales
False   216    216              216    216

In [17]:
# Final sales data
df.head()

Year  Month  ProductCategory   Sales
0  2009      1                1  1755.0
1  2009      1                2   524.0
2  2009      1                3   936.0
3  2009      2                1  1729.0
4  2009      2                2   496.0

In [18]:
df.shape

(216, 4)

# Preprocessing weather data

In [19]:
cdf_weather.head()

Year Month  Day Temp high (°C) Temp avg (°C) Temp low (°C)  \
0  2009   Jan    1             -3            -6            -9   
1  2009   Jan    2              1            -2            -5   
2  2009   Jan    3              3             1            -2   
3  2009   Jan    4              6             1            -4   
4  2009   Jan    5              6             5             3   

  Dew Point high (°C) Dew Point avg (°C) Dew Point low (°C) Humidity (%) high  \
0                 -16                -17                -19                54   
1                  -3                 -7                -17                78   
2                  -5                 -9                -13                72   
3                 -10                -12                -13                55   
4                  -1                 -5                -16                62   

      ...      Sea Level Press. (hPa) avg Sea Level Press. (hPa) low  \
0     ...                            1023                       1015   
1     ...                            1012                       1007   
2     ...                            1015                       1008   
3     ...                            1017                       1015   
4     ...                            1014                       1013   

  Visibility (km) high Visibility (km) avg Visibility (km) low  \
0                   16                  16                  16   
1                   16                  13                   2   
2                   16                  16                  16   
3                   16                  16                  16   
4                   16                  16                  16   

  Wind (km/h) low Wind (km/h) avg Wind (km/h) high Precip. (mm) sum  \
0              37              18               60                0   
1              27              10               48                T   
2              27              16               42                T   
3              32              12               40                0   
4              23              11               34                T   

  WeatherEvent  
0          NaN  
1         Snow  
2          NaN  
3          NaN  
4          NaN  

[5 rows x 23 columns]

In [20]:
cdf_weather.fillna(value = 'Normal', inplace = True)

In [21]:
cdf_weather.replace(to_replace= '-', value= np.NaN , inplace= True)


In [22]:
cdf_weather.replace(to_replace= 'T', value= np.NaN , inplace= True)

In [23]:
cdf_weather.isna().apply(pd.value_counts)

Year   Month     Day  Temp high (°C)  Temp avg (°C)  Temp low (°C)  \
False  2922.0  2922.0  2922.0            2920           2920           2920   
True      NaN     NaN     NaN               2              2              2   

       Dew Point high (°C)  Dew Point avg (°C)  Dew Point low (°C)  \
False                 2920                2920                2920   
True                     2                   2                   2   

       Humidity (%) high      ...       Sea Level Press. (hPa) avg  \
False               2920      ...                             2903   
True                   2      ...                               19   

       Sea Level Press. (hPa) low  Visibility (km) high  Visibility (km) avg  \
False                        2903                  2894                 2894   
True                           19                    28                   28   

       Visibility (km) low  Wind (km/h) low  Wind (km/h) avg  \
False                 2894             2902             2902   
True                    28               20               20   

       Wind (km/h) high  Precip. (mm) sum  WeatherEvent  
False              2863              2716        2922.0  
True                 59               206           NaN  

[2 rows x 23 columns]

In [24]:
cdf_weather = cdf_weather.interpolate()

In [25]:
cdf_weather.isna().apply(pd.value_counts)

Year  Month   Day  Temp high (°C)  Temp avg (°C)  Temp low (°C)  \
False  2922   2922  2922            2922           2922           2922   

       Dew Point high (°C)  Dew Point avg (°C)  Dew Point low (°C)  \
False                 2922                2922                2922   

       Humidity (%) high      ...       Sea Level Press. (hPa) avg  \
False               2922      ...                             2922   

       Sea Level Press. (hPa) low  Visibility (km) high  Visibility (km) avg  \
False                        2922                  2922                 2922   

       Visibility (km) low  Wind (km/h) low  Wind (km/h) avg  \
False                 2922             2922             2922   

       Wind (km/h) high  Precip. (mm) sum  WeatherEvent  
False              2922              2922          2922  

[1 rows x 23 columns]

In [26]:
cdf_weather.Month.replace( to_replace= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'] , value = [1,2,3,4,5,6,7,8,9,10,11,12], inplace = True)

# Only with month-wise sales data

In [27]:
df.head()

Year  Month  ProductCategory   Sales
0  2009      1                1  1755.0
1  2009      1                2   524.0
2  2009      1                3   936.0
3  2009      2                1  1729.0
4  2009      2                2   496.0

In [28]:
X = df.iloc[:,0:3]
y = df.Sales

In [64]:
DT = DecisionTreeRegressor(max_depth= 8, random_state= 6)
DT.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=6, splitter='best')

In [65]:
predicted_labels = DT.predict(X)
rms = sqrt(mean_squared_error(y, predicted_labels))
rms

45.13822148977509

In [31]:
predicted_labels.shape

(216,)

# Validation (just for fun)


In [49]:
from sklearn.model_selection import train_test_split

X_trainv, X_testv, y_trainv, y_testv = train_test_split(df, y, test_size = 0.30, random_state = 21)

In [66]:
DTval = DecisionTreeRegressor(max_depth= 8,min_samples_leaf = 1, random_state= 5)
DTval.fit(X_trainv,y_trainv)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=5, splitter='best')

In [67]:
pred_val = DTval.predict(X_testv)
rms_val = sqrt(mean_squared_error(y_testv, pred_val))
rms_val

45.37836658415719

# Test

In [68]:
test = pd.read_csv('test.csv')

In [75]:
test.head()

Year  Month  ProductCategory  Sales
0  2015      1                1    NaN
1  2015      1                2    NaN
2  2015      1                3    NaN
3  2015      2                1    NaN
4  2015      2                2    NaN

In [70]:
test.rename(columns = {'Sales(In ThousandDollars)' : 'Sales'}, inplace= True)

In [71]:
test['ProductCategory'] = test['ProductCategory'].astype('category', categories = test['ProductCategory'].unique( ))

C:\Users\noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
Year               36 non-null int64
Month              36 non-null int64
ProductCategory    36 non-null category
Sales              0 non-null float64
dtypes: category(1), float64(1), int64(2)
memory usage: 1.1 KB


In [73]:
test['ProductCategory'].replace(to_replace =  ['OtherClothing','MenClothing','WomenClothing'] , value = [3,2,1], inplace = True)

In [74]:
X_test = test.iloc[:,0:3]
y_test = test.Sales

In [76]:
pred = DT.predict(X_test)
# RMS = sqrt(mean_squared_error(y_test,pred))
pred

array([2961.        ,  573.        , 1147.5       , 3381.        ,
        636.        , 1147.5       , 4245.5       ,  669.        ,
       1241.66666667, 4245.5       ,  748.16666667, 1241.66666667,
       4421.        ,  748.16666667, 1241.66666667, 3819.        ,
        669.77777778, 1241.66666667, 3819.        ,  669.77777778,
       1241.66666667, 4067.        ,  669.77777778, 1241.66666667,
       4481.        ,  697.5       , 1056.        , 4434.        ,
        771.        , 1139.5       , 4525.        ,  800.5       ,
       1468.        , 5664.        , 1070.        , 1967.        ])

In [77]:
result_df = pd.DataFrame({'Sales' : pred[:,]})
result_df.to_csv('C:\\Users\\noah\\Desktop\\Unsupervised learning\\Wallmart case study\\result6.csv')

# Using linear regression

In [78]:
reg = LinearRegression().fit(X, y)


In [79]:
r = reg.predict(X_test)
r

array([2965.99738248, 1874.28557692,  782.57377137, 3018.7954594 ,
       1927.08365385,  835.37184829, 3071.59353632, 1979.88173077,
        888.16992521, 3124.39161325, 2032.67980769,  940.96800214,
       3177.18969017, 2085.47788462,  993.76607906, 3229.98776709,
       2138.27596154, 1046.56415598, 3282.78584402, 2191.07403846,
       1099.36223291, 3335.58392094, 2243.87211538, 1152.16030983,
       3388.38199786, 2296.67019231, 1204.95838675, 3441.18007479,
       2349.46826923, 1257.75646368, 3493.97815171, 2402.26634615,
       1310.5545406 , 3546.77622863, 2455.06442308, 1363.35261752])

In [80]:
result_df = pd.DataFrame({'Sales' : r[:,]})
result_df.to_csv('C:\\Users\\noah\\Desktop\\Unsupervised learning\\Wallmart case study\\result4.csv')